<a href="https://colab.research.google.com/github/elizagb/Google-Earth-Engine-Practice/blob/main/Combo_Label_QGIS_FeatureCollection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import ee

import json
import geopandas as gpd

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
# Trigger the authentication flow.
ee.Authenticate()

True

In [10]:
# start up the session
ee.Initialize(project='ee-egblack')

In [11]:
# Read the GeoJSON using geopandas
# gdf = gpd.read_file('/content/training_polygons.geojson')
file_path = "/content/drive/MyDrive/Colab-Notebooks/training_polygons.geojson"


with open(file_path, 'r') as f:
  geojson_data = json.load(f)

print(geojson_data)

{'type': 'FeatureCollection', 'name': 'training_polygons', 'crs': {'type': 'name', 'properties': {'name': 'urn:ogc:def:crs:OGC:1.3:CRS84'}}, 'features': [{'type': 'Feature', 'properties': {'fid': 1, 'cover-type': 'water-body', 'floral-abundance': None, 'fire-severity': None, 'tree-cover': None, 'artificial-type': None}, 'geometry': {'type': 'Polygon', 'coordinates': [[[-122.57915429851695, 44.148703305109585], [-122.57874394034084, 44.148853414717834], [-122.57744044966373, 44.147242603221144], [-122.5766921494602, 44.14613983595256], [-122.57714273883008, 44.14600126688067], [-122.57915429851695, 44.148703305109585]]]}}, {'type': 'Feature', 'properties': {'fid': 2, 'cover-type': 'water-body', 'floral-abundance': None, 'fire-severity': None, 'tree-cover': None, 'artificial-type': None}, 'geometry': {'type': 'Polygon', 'coordinates': [[[-122.24223979834892, 44.125865379171515], [-122.23953888939586, 44.12642937503368], [-122.23747637710444, 44.116805457724055], [-122.22519951822709, 44.

In [12]:
# Define the Holiday Farm Fire center
center = ee.Geometry.Point([-122.418960, 44.163945])

# Define a region around the center (e.g., a 20 km buffer)
region = center.buffer(20000)  # 20 km radius

In [13]:
# Load Sentinel-2 surface reflectance imagery for the post-fire period
# image = (
#     ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
#     .filterBounds(region)
#     .filterDate("2024-03-01", "2024-09-30")
#     .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 20))  # Filter low-cloud images
#     .median()  # Take median composite to reduce noise
#     .clip(region)
# )


# Filtered collections for different years
image_2023 = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
    .filterBounds(region) \
    .filterDate("2023-03-01", "2023-09-30") \
    .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 10))

image_2024 = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
    .filterBounds(region) \
    .filterDate("2024-03-01", "2024-09-30") \
    .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 10))

# TODO: try with 20% could cover

# Merge years into one collection
image_combined = image_2023.merge(image_2024)

# Build composite image
image = image_combined.median().clip(region)


In [14]:
def encode_training_feature(feature):
    def encode(field, mapping):
        val = feature.get(field)
        # Use ee.String(val).toLowerCase() for server-side lowercase conversion
        return ee.Algorithms.If(
            val,
            ee.Dictionary(mapping).get(ee.String(val).toLowerCase(), -1),
            -1
        )

    # Individual encodings
    floral = encode('floral-abundance', {
        'low': 1, 'high': 2, 'null': 0
    })

    fire = encode('fire-severity', {
        'low': 1, 'medium': 2, 'high': 3, 'null': 0
    })

    tree = encode('tree-cover', {
        'thin': 1, 'thick': 2, 'null': 0
    })

    artificial = encode('artificial-type', {
        'road-gravel': 1, 'road-paved': 2, 'other': 3, 'null': 0
    })

    cover = encode('cover-type', {
        'salvage-logged': 1,
        'forest-dougfir': 2,
        'forest-mixed': 3,
        'agriculture': 4,
        'artificial': 5,
        'water-body': 6,
        'bare-rock': 7,
        'grassland': 8,
        'null': 0
    })

    # Create a composite class ID — cast to ee.Number
    # Current structure: combo_class = cover*10000 + fire*1000 + tree*100 + floral*10 + artificial
    combo_class = (
        ee.Number(cover).multiply(10000)
        .add(ee.Number(fire).multiply(1000))
        .add(ee.Number(tree).multiply(100))
        .add(ee.Number(floral).multiply(10))
        .add(ee.Number(artificial))
    )

    return feature.set({
        'floral_encoded': floral,
        'fire_encoded': fire,
        'tree_encoded': tree,
        'artificial_encoded': artificial,
        'cover_encoded': cover,
        'combo_class': combo_class
    })


In [15]:
training_fc = ee.FeatureCollection(geojson_data)

# Create encoded training feature collection
training_fc_encoded = training_fc.map(encode_training_feature)

In [16]:
# Confirms that the training areas plot correctly and overlap with the sentinel 2 image region

import geemap
Map = geemap.Map()
Map.centerObject(region, 11)  # Center the map to your region

# Add the image to the map
Map.addLayer(image, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}, 'Image') # Visualize with RGB bands

# Add the training polygons
Map.addLayer(training_fc_encoded, {}, 'Training Data')

Map

Map(center=[44.16396551764556, -122.41895920682659], controls=(WidgetControl(options=['position', 'transparent…

In [17]:
band_names = image.bandNames().getInfo()

In [18]:
training_data = image.sampleRegions(
    collection=training_fc_encoded,
    properties=['combo_class'],
    scale=10,
    geometries=True
)

In [19]:
trained_classifier = ee.Classifier.smileRandomForest(numberOfTrees=100).train(
    features=training_data,
    classProperty='combo_class',
    inputProperties=band_names
)

# IN PROGRESS: using probability feature to create a confidence interval
# trained_classifier = ee.Classifier.smileRandomForest(numberOfTrees=100)
# trained_classifier_prob = trained_classifier.setOutputMode('PROBABILITY').train(
#     features=training_data,
#     classProperty='cover-type',
#     inputProperties=band_names
# )

# Classify the image using the trained classifier
classified_image = image.classify(trained_classifier)



In [20]:
import random

# Get combo classes from training data
combo_classes = training_fc_encoded.aggregate_array('combo_class').distinct().getInfo()
combo_classes = sorted([int(c) for c in combo_classes])  # ensure numeric order

# Create a random color hex code for each class
def random_color():
    return "#{:06x}".format(random.randint(0, 0xFFFFFF))

palette = [random_color() for _ in combo_classes]

print("Combo classes:", combo_classes)
print("Palette:", palette)


Map.addLayer(
    classified_image,
    {
        'min': min(combo_classes),
        'max': max(combo_classes),
        'palette': palette
    },
    'Classified Image'
)
Map

Combo classes: [8909, 8919, 38889, 48891, 48892, 48893, 58889]
Palette: ['#77ffad', '#e02d85', '#b8574b', '#8257f4', '#b02e44', '#85386d', '#7b5e69']


Map(center=[44.16396551764556, -122.41895920682659], controls=(WidgetControl(options=['position', 'transparent…

In [21]:
# Check unique class labels / the number of each
class_labels = training_fc_encoded.aggregate_histogram('combo_class').getInfo()
print("Combo class histogram:", class_labels)

Combo class histogram: {'38889': 3, '48891': 2, '48892': 2, '48893': 5, '58889': 3, '8909': 1, '8919': 1}


In [22]:

# In progress
# Encoding strcuture: combo_class = cover*10000 + fire*1000 + tree*100 + floral*10 + artificial

def decode_combo_class(combo_id):
    cover = (combo_id // 10000) % 10
    fire = (combo_id // 1000) % 10
    tree = (combo_id // 100) % 10
    floral = (combo_id // 10) % 10
    artificial = combo_id % 10

    cover_map = {
        0:'null',
        1:'salvage-logged',
        2:'forest-dougfir',
        3:'forest-mixed',
        4:'agriculture',
        5:'artificial',
        6:'water-body',
        7:'bare-rock',
        8:'grassland',
    }
    fire_map = {
        0:'null',
        1:'low',
        2:'medium',
        3:'high',
    }
    tree_map = {
        0:'null',
        1:'thin',
        2:'thick',
    }
    floral_map = {
        0:'null',
        1:'low',
        2:'high',
    }
    artificial_map = {
        0:'null',
        1:'road-gravel',
        2:'road-paved',
        3:'other',
    }

    return (
        f"Cover-type: {cover_map.get(cover, 'unknown')}, "
        f"Fire-severity: {fire_map.get(fire, 'unknown')}, "
        f"Tree-cover: {tree_map.get(tree, 'unknown')}, "
        f"Floral-abundance: {floral_map.get(floral, 'unknown')}, "
        f"Artificial-type: {artificial_map.get(artificial, 'unknown')}"
    )

